In [ ]:
!pip install PyMuPDF
!pip install pytesseract
!sudo apt install tesseract-ocr
!pip install pyffx

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install pdfplumber

In [ ]:
import re
import fitz
import pytesseract
from PIL import Image
import io

In [ ]:
# def extract_text_from_pdf(pdf_path):
#     """Extracts text from a PDF file, including text from images."""
#     doc = fitz.open(pdf_path)
#     text = ""

#     for page in doc:
#         text += page.get_text("text") + "\n"

#         # Extract images and perform OCR
#         for img_index, img in enumerate(page.get_images(full=True)):
#             xref = img[0]
#             base_image = doc.extract_image(xref)
#             image_bytes = base_image["image"]
#             image = Image.open(io.BytesIO(image_bytes))
#             text += pytesseract.image_to_string(image) + "\n"
#     return text


In [ ]:
import pdfplumber

def extract_text_from_pdf(filepath):
    text = ""
    with pdfplumber.open(filepath) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text


In [ ]:
# text = extract_text_from_pdf("/content/1.pdf")

In [ ]:
# print(text)

In [ ]:
import pyffx
import random
import string

# secret_key = b'sup6meowkeycat'
secret_key = b'sbh86'
# Mapping for deterministic pseudonymization
pseudonym_map = {}

def get_alpha_encryptor(length):
    return pyffx.String(secret_key, alphabet=string.ascii_letters + ' ', length=length)

def get_upper_encryptor(length):
    return pyffx.String(secret_key, alphabet=string.ascii_uppercase + ' ', length=length)

def get_numeric_encryptor(length):
    return pyffx.Integer(secret_key, length=length)

def fpe_encrypt(value):
    if value in pseudonym_map:
        return pseudonym_map[value]

    try:
        if value.isdigit():
            enc = get_numeric_encryptor(len(value))
            encrypted = str(enc.encrypt(int(value))).zfill(len(value))

        elif re.match(r'^\d{2}/\d{2}/\d{2,4}$', value):  # Dates
            parts = value.split('/')
            encrypted_parts = [
                str(get_numeric_encryptor(len(p)).encrypt(int(p))).zfill(len(p))
                for p in parts
            ]
            encrypted = '/'.join(encrypted_parts)

        else:
            # Dynamically extract all unique characters from value
            alphabet = ''.join(sorted(set(value)))
            enc = pyffx.String(secret_key, alphabet=alphabet, length=len(value))
            encrypted = enc.encrypt(value)

    except Exception as e:
        print(f"[!] FPE error on '{value}': {e}")
        encrypted = value

    pseudonym_map[value] = encrypted
    return encrypted

In [ ]:
# # Anonymisation : encryptage non reversible, ni reproductible
# import hashlib
# import random
# import string
# import re

# def random_pseudo(length=12):
#     return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

# def fpe_anonymize(value):
#     if value in pseudonym_map:
#         return pseudonym_map[value]

#     try:
#         pseudo = random_pseudo()
#     except Exception as e:
#         print(f"[!] Anonymization error on '{value}': {e}")
#         pseudo = "ANON"

#     pseudonym_map[value] = pseudo
#     return pseudo


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import CamembertTokenizer, CamembertForTokenClassification
model_path = "/content/drive/MyDrive/ner_model/camembert-ner"
model = CamembertForTokenClassification.from_pretrained(model_path)
tokenizer = CamembertTokenizer.from_pretrained(model_path)


In [ ]:
from transformers import pipeline
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=device,
)

Device set to use cpu


In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


def normalize_whitespace(s):
    # Collapse all whitespace (spaces, tabs, newlines) into a single space
    return re.sub(r'\s+', ' ', s).strip()

def encrypt_ner_entities_chunked(text):
    sentences = sent_tokenize(text, language="french")
    encrypted_map = {}

    # Work on a normalized version of the text for better matching
    normalized_text = normalize_whitespace(text)

    for sentence in sentences:
        entities = ner_pipeline(sentence)
        for entity in entities:
            if entity["entity_group"] in ["PER"]:
                original = normalize_whitespace(entity["word"])
                if original not in encrypted_map:
                    encrypted_map[original] = fpe_encrypt(original)

    for original, encrypted in encrypted_map.items():
        pattern = r'\b' + r'\s*'.join(map(re.escape, original.split())) + r'\b'
        text = re.sub(pattern, encrypted, text, flags=re.IGNORECASE)

    return text



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# # Anonymize rather than pseudonymise
# def normalize_whitespace(s):
#     """Collapse all whitespace (spaces, tabs, newlines) into a single space"""
#     return re.sub(r'\s+', ' ', s).strip()

# def random_pseudo(length=12):
#     """Generate a random anonymized string"""
#     return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

# def anonymize_ner_entities_chunked(text, ner_pipeline):
#     """
#     - Tokenize text into sentences
#     - Detect NER entities
#     - Replace each PER, LOC, ORG with a non-deterministic pseudo-ID
#     """
#     sentences = sent_tokenize(text, language="french")
#     anonymize_map = {}

#     # Work on a normalized version for matching
#     normalized_text = normalize_whitespace(text)

#     for sentence in sentences:
#         entities = ner_pipeline(sentence)
#         for entity in entities:
#             if entity["entity_group"] in ["PER", "LOC", "ORG"]:
#                 original = normalize_whitespace(entity["word"])
#                 if original not in anonymize_map:
#                     anonymize_map[original] = random_pseudo()

#     # Remplacer les occurrences avec cohérence intra-document
#     for original, pseudo in anonymize_map.items():
#         # Construire un pattern robuste qui ignore les espaces multiples
#         pattern = r'\b' + r'\s*'.join(map(re.escape, original.split())) + r'\b'
#         text = re.sub(pattern, pseudo, text, flags=re.IGNORECASE)

#     return text

In [ ]:
def encrypt_doc(doc_path, output_folder):

  text = extract_text_from_pdf(doc_path)

  # names, locations, organizations
  text = encrypt_ner_entities_chunked(text)

  # Codes
  text = re.sub(r'\b(?:\d{8,}|\d{6,}[A-Z]\d{3,})\b', lambda m: fpe_encrypt(m.group()), text)

  # Dates
  patterns = [
          r'(?i)\b(date de naissance[:\s]*)\d{2}/\d{2}/\d{2,4}',  # Format numérique
          r'(?i)\b(né\(e\)? le[:\s]*)\d{2}/\d{2}/\d{2,4}',         # Format numérique avec "né(e) le"
          r'(?i)\b(date de naissance[:\s]*)\d{1,2}\s+(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+\d{4}',
          r'(?i)\b(né\(e\)? le[:\s]*)\d{1,2}\s+(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+\d{4}'
      ]

  for pattern in patterns:
      text = re.sub(pattern, lambda m: m.group(1) + fpe_encrypt(m.group(0).replace(m.group(1), '')), text)

  # Email address
  text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', lambda m: fpe_encrypt(m.group()), text)

  # French phone numbers
  text = re.sub(r'\b(?:0\d(?:[ .-]?\d{2}){4})\b', lambda m: fpe_encrypt(m.group()), text)

  # Postal codes and addresses
  text = re.sub(r'\b\d{5} [A-Z]+\b', lambda m: fpe_encrypt(m.group()), text)

  # save
  base_name = os.path.basename(doc_path).replace('.pdf', '')
  output_filename = base_name + '_encrypted.txt'

  # Build the full output path in the desired folder
  output_path = os.path.join(output_folder, output_filename)

  # Write the output
  with open(output_path, "w", encoding="utf-8") as f:
      f.write(text)
  return text


In [ ]:
# def anonymise_doc(input_path, output_path, ner_pipeline):

#   text = extract_text_from_pdf(input_path)

#   # names, locations, organizations
#   text = anonymize_ner_entities_chunked(text, ner_pipeline)

#   # Codes
#   text = re.sub(r'\b(?:\d{8,}|\d{6,}[A-Z]\d{3,})\b', lambda m: fpe_anonymize(m.group()), text)

#   # Dates
#   text = re.sub(r'\b\d{2}/\d{2}/\d{2,4}\b', lambda m: fpe_anonymize(m.group()), text)
#   written_date_pattern = r'\b(?:le\s+)?(?:lundi|mardi|mercredi|jeudi|vendredi|samedi|dimanche)?\s*\d{1,2}\s+(?:janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+\d{4}\b'
#   text = re.sub(written_date_pattern, lambda m: fpe_anonymize(m.group()), text, flags=re.IGNORECASE)

#   # Email address
#   text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', lambda m: fpe_anonymize(m.group()), text)

#   # French phone numbers
#   text = re.sub(r'\b(?:0\d(?:[ .-]?\d{2}){4})\b', lambda m: fpe_anonymize(m.group()), text)

#   # Postal codes and addresses
#   text = re.sub(r'\b\d{5} [A-Z]+\b', lambda m: fpe_anonymize(m.group()), text)

#   # save
#   # output_path = doc_path.replace('.pdf', '_encrypted.txt')
#   with open(output_path, "w", encoding="utf-8") as f:
#       f.write(text)
#       print(f"✅ {output_path} créé")
#   return text


Example use


In [ ]:
doc_path = "/content/3.pdf"

In [ ]:
encrypted_text = encrypt_doc(doc_path)

TypeError: encrypt_doc() missing 1 required positional argument: 'output_folder'

In [ ]:
import os

input_dir = "/content/sample_data/test/"
output_dir = "/content/drive/MyDrive/PrivateBoodTest/"

for filename in os.listdir(input_dir):
    if filename.endswith(".pdf"):
        doc_path = os.path.join(input_dir, filename)
        print(f"Processing: {doc_path}")
        encrypt_doc(doc_path, output_dir)


Processing: /content/sample_data/test/38.pdf
Processing: /content/sample_data/test/9.pdf
Processing: /content/sample_data/test/31.pdf
Processing: /content/sample_data/test/16.pdf
Processing: /content/sample_data/test/15.pdf
Processing: /content/sample_data/test/37.pdf
Processing: /content/sample_data/test/49.pdf
Processing: /content/sample_data/test/45.pdf
Processing: /content/sample_data/test/3.pdf
Processing: /content/sample_data/test/26.pdf
Processing: /content/sample_data/test/30.pdf
Processing: /content/sample_data/test/14.pdf
Processing: /content/sample_data/test/19.pdf
Processing: /content/sample_data/test/41.pdf
Processing: /content/sample_data/test/10.pdf
Processing: /content/sample_data/test/2.pdf
Processing: /content/sample_data/test/39.pdf
Processing: /content/sample_data/test/7.pdf
Processing: /content/sample_data/test/20.pdf
Processing: /content/sample_data/test/11.pdf
Processing: /content/sample_data/test/44.pdf
Processing: /content/sample_data/test/36.pdf
Processing: /c

In [ ]:
# import os

# input_dir = "/content/sample_data/test/"
# output_dir = "/content/drive/MyDrive/PrivateBoodTest/"

# os.makedirs(output_dir, exist_ok=True)

# n=51
# for i in range(1, n):
#     doc_filename = f"{i}.pdf"
#     doc_path = os.path.join(input_dir, doc_filename)
#     print(doc_path)

#     if os.path.exists(doc_path):
#         output_filename = f"{i}_encrypted.txt"
#         output_path = os.path.join(output_dir, output_filename)
#         print(output_path)
#         anonymise_doc(doc_path, output_path, ner_pipeline)
